<a href="https://colab.research.google.com/github/envest99/LeShop-Same-Day/blob/master/ENAEF_Same_Day_15112019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from ortools.sat.python import cp_model
#pip install --upgrade --user ortools
!python -m pip install --upgrade --user ortools
!python -m pip install -U --user ortools

In [0]:
!python -c "from ortools.linear_solver import pywraplp"

In [0]:
import os
import numpy as np
import math
import collections
import pandas as pd

In [0]:
def create_data_model():
  """Stores the data for the problem."""
  data = {}
  data['time_matrix'] = [
[0, 1, 3, 9, 10, 17, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 9, 10, 16, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 9, 10, 18, 6, 8, 12, 15, 8],
[1, 0, 6, 8, 5, 7, 15, 15, 9, 11, 4, 12, 9, 6, 11, 3, 7, 11, 16, 15, 9, 3, 17, 4, 15, 5, 6, 6, 15, 12, 4, 17, 5, 12, 13, 2, 4, 4, 8, 11, 5],
[3, 6, 0, 16, 11, 4, 14, 16, 4, 16, 5, 9, 2, 2, 13, 3, 14, 7, 10, 4, 3, 17, 3, 5, 15, 3, 17, 8, 8, 1, 12, 4, 12, 18, 16, 11, 6, 11, 1, 7, 2],
[9, 8, 16, 0, 8, 17, 18, 10, 10, 18, 12, 15, 6, 5, 18, 15, 7, 9, 9, 3, 15, 12, 2, 3, 1, 5, 6, 4, 10, 12, 1, 15, 18, 14, 1, 13, 9, 1, 9, 8, 13],
[10, 5, 11, 8, 0, 5, 7, 8, 9, 16, 13, 14, 9, 9, 15, 17, 2, 8, 16, 9, 16, 12, 3, 17, 3, 2, 13, 18, 15, 1, 13, 7, 3, 1, 9, 8, 1, 6, 8, 4, 18],
[18, 7, 4, 17, 5, 0, 11, 7, 13, 1, 6, 16, 7, 4, 2, 16, 11, 7, 12, 7, 1, 1, 17, 7, 12, 1, 3, 4, 15, 6, 13, 2, 4, 10, 11, 17, 1, 5, 4, 6, 11],
[6, 15, 14, 18, 7, 11, 0, 11, 6, 10, 6, 9, 4, 5, 14, 17, 8, 4, 17, 17, 9, 2, 14, 6, 11, 8, 11, 9, 8, 8, 4, 13, 2, 16, 15, 6, 8, 6, 7, 13, 2],
[8, 15, 16, 10, 8, 7, 11, 0, 12, 2, 7, 7, 2, 5, 11, 18, 12, 15, 6, 9, 2, 11, 16, 8, 1, 10, 6, 13, 15, 8, 4, 10, 6, 14, 10, 4, 2, 14, 6, 15, 12],
[12, 9, 4, 10, 9, 13,6, 12,0, 8, 8, 1, 7, 1, 6, 11, 14, 15, 15, 18, 12, 7, 18, 4, 16, 6, 17, 6, 6, 13, 8, 12, 1, 12, 5, 17, 2, 13, 15, 7, 14],
[15, 11, 16, 18, 16, 1, 10, 2, 8, 0, 5, 7, 7, 2, 17, 11, 1, 16, 12, 8, 11, 4, 8, 10, 18, 6, 1, 16, 14, 15, 17, 14, 10, 8, 10, 14, 8, 1, 18, 2, 11],
[8, 4, 5, 12, 13, 6, 6, 7, 8, 5, 0, 6, 9, 17, 7, 10, 15, 1, 1, 7, 3, 10, 5, 1, 3, 5, 18, 18, 10, 15, 1, 18, 10, 12, 15, 2, 5, 6, 10, 6, 6],
[13, 12, 9, 15, 14, 16, 9, 7, 1, 7, 6, 0, 9, 14, 9, 8, 6, 17, 11, 8, 6, 5, 18, 13, 3, 12, 6, 6, 7, 16, 11, 4, 6, 16, 17, 2, 10, 3, 8, 6, 1],
[9, 9, 2, 6, 9, 7, 4, 2, 7, 7, 9, 9, 0, 17, 2, 13, 1, 9, 6, 16, 8, 10, 11, 16, 15, 4, 2, 2, 15, 12, 10, 15, 16, 12, 9, 12, 6, 6, 2, 10, 4],
[13, 6, 2, 5, 9, 4, 5, 5, 1, 2, 17, 14, 17, 0, 12, 9, 12, 12, 10, 7, 4, 16, 17, 5, 1, 17, 13, 3, 6, 18, 3, 8, 9, 5, 10, 15, 13, 6, 3, 16, 3],
[3, 11, 13, 18, 15, 2, 14, 11, 6, 17, 7, 9, 2, 12, 0, 9, 6, 8, 1, 11, 8, 10, 16, 13, 1, 3, 18, 5, 4, 7, 11, 7, 2, 7, 5, 3, 5, 11, 4, 6, 13],
[4, 3, 3, 15, 17, 16,17, 18, 11, 11, 10, 8, 13, 9, 9, 0, 16, 6, 18, 6, 14, 9, 12, 5, 7, 14, 5, 15, 10, 12, 5, 8, 8, 13, 7, 15, 11, 18, 5, 1, 13],
[5, 7, 14,7, 2, 11, 8, 12, 14, 1, 15, 6, 1, 12, 6, 16, 0, 2, 15, 2, 6, 1, 10, 18, 17, 2, 7,9, 8, 5, 15, 13, 15, 9, 17, 16, 18, 9, 17, 6, 18],
[9, 11, 7, 9, 8, 7, 4, 15, 15, 16, 1, 17, 9, 12, 8, 6, 2, 0, 4, 18, 18, 9, 15, 16, 14, 17,7, 2, 12, 2, 17, 15, 18, 6, 16, 16, 1, 10, 11, 12, 2],
[9, 16, 10, 9, 16, 12, 17, 6, 15, 12, 1, 11, 6, 10, 1, 18, 15, 4, 0, 3, 4, 15, 10, 8, 3, 14, 16, 16, 11, 17, 3, 15, 1, 10, 2, 4, 8, 6, 1, 6, 17],
[10, 15, 4, 3, 9, 7, 17, 9, 18, 8, 7, 8, 16, 7, 11, 6, 2, 18, 3, 0, 15, 2, 14, 9, 6, 18, 12, 1, 18, 2, 2, 1, 9, 2, 12, 13, 11, 15, 13, 16, 13],
[18, 9, 3, 15, 16, 1, 9, 2, 12, 11, 3, 6, 8, 4, 8, 14, 6, 18, 4, 15, 0, 4, 9, 13, 13, 10, 15, 11, 6, 14, 6, 7, 13, 3, 12, 4, 4, 18, 3, 14, 9],
[6, 3, 17, 12, 12, 1, 2, 11, 7, 4, 10, 5, 10, 16, 10, 9, 1, 9, 15, 2, 4, 0, 18, 17, 1, 6, 10, 2, 8, 12, 6, 4, 18, 11, 1, 10, 7, 15, 13, 3, 17],
[8, 17, 3, 2, 3, 17, 14, 16, 18, 8, 5, 18, 11, 17, 16, 12, 10, 15, 10, 14, 9, 18, 0, 17, 17, 12, 5, 12, 16, 8, 17, 7, 12, 6, 5, 7, 4, 11, 1, 15, 16],
[12, 4, 5, 3, 17, 7, 6, 8, 4, 10, 1, 13, 16, 5, 13, 5, 18, 16, 8, 9, 13, 17, 17, 0, 4, 6, 4, 14, 15, 3, 14, 11, 9, 4, 9, 15, 11, 18, 2, 16, 2],
[15, 15, 15, 1, 3, 12, 11, 1, 16, 18, 3, 3, 15, 1, 1, 7, 17, 14, 3, 6, 13, 1, 17, 4, 0, 9, 18, 5, 2, 11, 12, 3, 13, 5, 2, 14, 9, 5, 18, 10, 2],
[8, 5, 3, 5, 2, 1, 8, 10, 6, 6, 5, 12, 4, 17, 3, 14, 2, 17, 14, 18, 10, 6, 12, 6, 9, 0, 12, 14, 1, 8, 15, 14, 8, 8, 18, 18, 1, 15, 1, 14, 3],
[13, 6, 17, 6, 13, 3, 11, 6, 17, 1, 18, 6, 2, 13, 18, 5, 7, 7, 16, 12, 15, 10, 5, 4, 18, 12, 0, 13, 2, 7, 8, 16, 3, 7, 16, 4, 2, 13, 9, 5, 8],
[9, 6, 8, 4, 18, 4, 9, 13, 6, 16, 18, 6, 2, 3, 5, 15, 9, 2, 16, 1, 11, 2, 12, 14, 5, 14, 13, 0, 4, 13, 8, 16, 10, 9, 5, 9, 18, 5, 11, 7, 1],
[13, 15, 8, 10, 15, 15, 8, 15, 6, 14, 10, 7, 15, 6, 4, 10, 8, 12, 11, 18, 6, 8, 16, 15, 2, 1, 2, 4, 0, 7, 16, 8, 14, 6, 8, 16, 6, 13, 9, 6, 3],
[3, 12, 1, 12, 1, 6, 8, 8, 13, 15, 15, 16, 12, 18, 7, 12, 5, 2, 17, 2, 14, 12, 8, 3, 11, 8, 7, 13, 7, 0, 1, 16, 18, 10, 2, 8, 12, 16, 2, 4, 13],
[4, 4, 12, 1, 13, 13, 4, 4, 8, 17, 1, 11, 10, 3, 11, 5, 15, 17, 3, 2, 6, 6, 17, 14, 12, 15, 8, 8, 16, 1, 0, 2, 9, 1, 18, 12, 12, 14, 2, 14, 16],
[5, 17, 4, 15, 7, 2, 13, 10, 12, 14, 18, 4, 15, 8, 7, 8, 13, 15, 15, 1, 7, 4, 7, 11, 3, 14, 16, 16, 8, 16, 2, 0, 11, 7, 14, 7, 7, 5, 18, 16, 6],
[9, 5, 12, 18, 3, 4, 2, 6, 1, 10, 10, 6, 16, 9, 2, 8, 15, 18, 1, 9, 13, 18, 12, 9, 13, 8, 3, 10, 14, 18, 9, 11, 0, 9, 9, 4, 10, 15, 2, 3, 15],
[9, 12, 18, 14, 1, 10, 16, 14, 12, 8, 12, 16, 12, 5, 7, 13, 9, 6, 10, 2, 3, 11, 6, 4, 5, 8, 7, 9, 6, 10, 1, 7, 9, 0, 4, 10, 18, 11, 14, 8, 15],
[10, 13, 16, 1, 9, 11, 15, 10, 5, 10, 15, 17, 9, 10, 5, 7, 17, 16, 2, 12, 12, 1, 5, 9, 2, 18, 16, 5, 8, 2, 18, 14, 9, 4, 0, 11, 11, 12, 18, 9, 3],
[18, 2, 11, 13, 8, 17, 6, 4, 17, 14, 2, 2, 12, 15, 3, 15, 16, 16, 4, 13, 4, 10, 7, 15, 14, 18, 4, 9, 16, 8, 12, 7, 4, 10, 11, 0, 1, 4, 10, 14, 9],
[6, 4, 6, 9, 1, 1, 8, 2, 2, 8, 5, 10, 6, 13, 5, 11, 18, 1, 8, 11, 4, 7, 4, 11, 9, 1, 2, 18, 6, 12, 12, 7, 10, 18, 11, 1, 0, 15, 5, 12, 8],
[8, 4, 11, 1, 6, 5, 6, 14, 13, 1, 6, 3, 6, 6, 11, 18, 9, 10, 6, 15, 18, 15, 11, 18, 5, 15, 13, 5, 13, 16, 14, 5, 15, 11, 12, 4, 15, 0, 13, 13, 18],
[12, 8, 1, 9, 8, 4, 7, 6, 15, 18, 10, 8, 2, 3, 4, 5, 17, 11, 1, 13, 3, 13, 1, 2, 18, 1, 9, 11, 9, 2, 2, 18, 2, 14, 18, 10, 5, 13, 0, 3, 15],
[15, 11, 7, 8, 4, 6, 13, 15, 7, 2, 6, 6, 10, 16, 6, 1, 6, 12, 6, 16, 14, 3, 15, 16, 10, 14, 5, 7, 6, 4, 14, 16, 3, 8, 9, 14, 12, 13, 3, 0, 13],
[8, 5, 2, 13, 18, 11, 2, 12, 14, 11, 6, 1, 4, 3, 13, 13, 18, 2, 17, 13, 9, 17, 16, 2, 2, 3, 8, 1, 3, 13, 16, 6, 15, 15, 3, 9, 8, 18, 15, 13, 0]
  ]
  data['time_windows'] = [
      (0, 5),  # depot
      (7, 12),  # 1
      (10, 15),  # 2
      (16, 18),  # 3
      (10, 13),  # 4
      (0, 5),  # 5
      (5, 10),  # 6
      (0, 4),  # 7
      (5, 10),  # 8
      (0, 3),  # 9
      (10, 16),  # 10
      (10, 15),  # 11
      (0, 5),  # 12
      (5, 10),  # 13
      (7, 8),  # 14
      (10, 15),  # 15
      (11, 15),  # 16
  ]
  data['num_vehicles'] = 4
  data['depot'] = 0
  return data

  time = 'Time'
  routing.AddDimension(
     transit_callback_index,
     30,  # allow waiting time
     100,  # maximum time per vehicle
     False,  # Don't force start cumul to zero.
     time)